# 🔧 Data Preprocessing for F1 Probability Modeling
## Preparing Singapore GP data for VAE and Bayesian Network implementation

This notebook handles:
- Feature engineering and selection
- Data normalization and scaling
- Missing value treatment
- Train/validation splits
- Feature encoding for ML models

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.impute import SimpleImputer, KNNImputer
import warnings
import glob
import os
from datetime import datetime
import pickle
import json

# Import configuration and feature weights
from config import *

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)

print("🔧 Preprocessing libraries loaded successfully!")
print(f"⚖️ Feature weighting system enabled: {len(get_weighted_features('all'))} weighted features")
print(f"🎯 Target circuit: {DATA_CONFIG['selected_circuit']}")
print(f"📊 Available circuits: {len(get_available_circuits())}")

🔧 Preprocessing libraries loaded successfully!
⚖️ Feature weighting system enabled: 15 weighted features
🎯 Target circuit: Singapore
📊 Available circuits: 23


    # 📂 Load Processed Dataset - Universal Circuit Support

## ✅ Data Flow Verification

Verify that we're correctly receiving data from notebook 02 (data analysis output)

In [2]:
# Load the cleaned dataset from notebook 02 (data analysis output)
print("📂 Loading cleaned dataset from analysis (notebook 02)...")

# Priority 1: Load cleaned data from notebook 02
cleaned_files = glob.glob('data/processed/*_cleaned_*.csv')

if cleaned_files:
    latest_file = max(cleaned_files, key=os.path.getctime)
    circuit_name = os.path.basename(latest_file).split('_')[0].upper()
    print(f"✅ Loading {circuit_name} GP cleaned data from notebook 02: {latest_file}")
    df = pd.read_csv(latest_file)
    data_source = "cleaned"
    
elif glob.glob('data/raw/*_prediction_weighted_*.csv'):
    # Fallback to weighted prediction data if cleaned not available
    weighted_files = glob.glob('data/raw/*_prediction_weighted_*.csv')
    latest_file = max(weighted_files, key=os.path.getctime)
    circuit_name = os.path.basename(latest_file).split('_')[0].upper()
    print(f"⚠️ Cleaned data not found, using weighted prediction data: {latest_file}")
    df = pd.read_csv(latest_file)
    data_source = "weighted_prediction"
else:
    print("❌ No data files found. Please run notebook 01 and 02 first.")
    df = pd.DataFrame()
    data_source = None

if not df.empty:
    print(f"✅ Data loaded successfully!")
    print(f"📊 Shape: {df.shape}")
    print(f"🏁 Years: {sorted(df['year'].unique())}")
    print(f"🏎️ Unique drivers: {df['driver_name'].nunique()}")
    print(f"🏁 Unique teams: {df['team'].nunique()}")
    
    # Check for weighted data features
    if 'data_weight' in df.columns:
        print(f"⚖️ Weighted dataset detected (weight range: {df['data_weight'].min():.1f} - {df['data_weight'].max():.1f})")
    
    print(f"🔧 Ready for preprocessing...")
else:
    print("🚫 Cannot proceed without data. Please run notebooks 01 and 02 first.")

📂 Loading cleaned dataset from analysis (notebook 02)...
✅ Loading SINGAPORE GP cleaned data from notebook 02: data/processed\singapore_cleaned_20251026_234118.csv
✅ Data loaded successfully!
📊 Shape: (240, 21)
🏁 Years: [np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
🏎️ Unique drivers: 30
🏁 Unique teams: 13
⚖️ Weighted dataset detected (weight range: 1.2 - 3.0)
🔧 Ready for preprocessing...


In [3]:
if not df.empty:
    print("✅ DATA FLOW VERIFICATION FROM NOTEBOOK 02")
    print("=" * 60)
    
    # Verify expected columns from notebook 02
    expected_base_columns = [
        'year', 'driver_name', 'team', 'grid_pos', 'finish_pos', 
        'quali_pos', 'points', 'pit_stops', 'pos_change'
    ]
    
    missing_columns = [col for col in expected_base_columns if col not in df.columns]
    if missing_columns:
        print(f"⚠️ Missing expected columns: {missing_columns}")
    else:
        print(f"✅ All expected base columns present")
    
    # Check for data weighting (from notebook 01)
    if 'data_weight' in df.columns:
        print(f"✅ Weighted data detected (from notebook 01)")
        print(f"   Weight range: {df['data_weight'].min():.1f} - {df['data_weight'].max():.1f}")
    
    # Check data quality (from notebook 02 cleaning)
    missing_pct = (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100
    print(f"📊 Data completeness: {100 - missing_pct:.1f}%")
    
    # Verify data source
    print(f"\n📁 Data source: {data_source}")
    print(f"📊 Shape: {df.shape}")
    print(f"🏁 Years: {sorted(df['year'].unique())}")
    
    print("\n✅ Data flow from notebook 02 verified successfully!")
    print("🔧 Ready to proceed with preprocessing...")
else:
    print("❌ Data flow verification failed - no data loaded")

✅ DATA FLOW VERIFICATION FROM NOTEBOOK 02
✅ All expected base columns present
✅ Weighted data detected (from notebook 01)
   Weight range: 1.2 - 3.0
📊 Data completeness: 100.0%

📁 Data source: cleaned
📊 Shape: (240, 21)
🏁 Years: [np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]

✅ Data flow from notebook 02 verified successfully!
🔧 Ready to proceed with preprocessing...


## 🎯 Feature Engineering

In [4]:
if not df.empty:
    print("📈 Creating rolling features and engineered metrics...")
    
    # Create a copy and sort by driver and year
    df_features = df.copy().sort_values(['driver_name', 'year'])
    
    # 1. Driver Form - Rolling averages (last 2 races per driver)
    print("  🏎️ Computing driver form metrics...")
    df_features['driver_quali_form'] = df_features.groupby('driver_name')['quali_pos'].rolling(
        window=2, min_periods=1
    ).mean().reset_index(0, drop=True)
    
    df_features['driver_race_form'] = df_features.groupby('driver_name')['finish_pos'].rolling(
        window=2, min_periods=1
    ).mean().reset_index(0, drop=True)
    
    df_features['driver_points_form'] = df_features.groupby('driver_name')['points'].rolling(
        window=2, min_periods=1
    ).mean().reset_index(0, drop=True)
    
    # 2. Team Performance
    print("  🏁 Computing team performance metrics...")
    team_quali_avg = df_features.groupby(['team', 'year'])['quali_pos'].transform('mean')
    df_features['team_quali_avg'] = team_quali_avg
    
    team_race_avg = df_features.groupby(['team', 'year'])['finish_pos'].transform('mean')
    df_features['team_race_avg'] = team_race_avg
    
    # 3. Strategic Features
    print("  ⛽ Computing strategy metrics...")
    df_features['quali_race_delta'] = df_features['finish_pos'] - df_features['quali_pos']
    df_features['abs_pos_change'] = abs(df_features['pos_change'])
    
    # 4. Normalize qualifying times
    print("  ⏱️ Normalizing qualifying times...")
    if 'gap_to_pole' in df_features.columns:
        year_max_gap = df_features.groupby('year')['gap_to_pole'].transform('max')
        df_features['gap_to_pole_normalized'] = (
            df_features['gap_to_pole'] / year_max_gap.replace(0, 1)
        ).fillna(0)
    
    # 5. Experience metrics
    print("  📚 Computing experience metrics...")
    df_features['cumulative_races'] = df_features.groupby('driver_name').cumcount() + 1
    df_features['years_experience'] = df_features.groupby('driver_name')['year'].transform('nunique')
    
    print(f"✅ Created {len(df_features.columns) - len(df.columns)} rolling/engineered features")
    print(f"📋 Updated dataset shape: {df_features.shape}")
else:
    print("⚠️ No data available for feature engineering.")

📈 Creating rolling features and engineered metrics...
  🏎️ Computing driver form metrics...
  🏁 Computing team performance metrics...
  ⛽ Computing strategy metrics...
  ⏱️ Normalizing qualifying times...
  📚 Computing experience metrics...
✅ Created 10 rolling/engineered features
📋 Updated dataset shape: (240, 31)


In [5]:
if not df.empty:
    print("🎯 Creating derived features...")
    
    # Ensure we have df_features from previous step
    if 'df_features' not in locals():
        df_features = df.copy()
    
    # 1. Driver skill rating
    print("  🏆 Computing driver skill ratings...")
    driver_stats = df_features.groupby('driver_name').agg({
        'points': 'mean',
        'finish_pos': 'mean',
        'pos_change': 'mean'
    })
    
    # Normalize driver skill (0-1 scale)
    driver_stats['skill_score'] = (
        (driver_stats['points'] / 25) * 0.4 +
        ((21 - driver_stats['finish_pos']) / 20) * 0.4 +
        ((driver_stats['pos_change'] + 10) / 20) * 0.2
    ).clip(0, 1)
    
    df_features['driver_skill'] = df_features['driver_name'].map(driver_stats['skill_score'])
    
    # 2. Team strength rating
    print("  🏁 Computing team strength ratings...")
    team_stats = df_features.groupby('team').agg({
        'points': 'mean',
        'finish_pos': 'mean'
    })
    
    max_team_points = team_stats['points'].max() if team_stats['points'].max() > 0 else 1
    team_stats['strength_score'] = (
        (team_stats['points'] / max_team_points) * 0.6 +
        ((21 - team_stats['finish_pos']) / 20) * 0.4
    ).clip(0, 1)
    
    df_features['team_strength'] = df_features['team'].map(team_stats['strength_score'])
    
    # 3. Strategy indicators
    print("  🔧 Computing strategy features...")
    df_features['is_aggressive_strategy'] = (df_features['pit_stops'] >= 2).astype(int)
    
    # Race completion rate
    max_laps_by_year = df_features.groupby('year')['total_laps'].transform('max')
    df_features['race_completion_rate'] = (
        df_features['total_laps'] / max_laps_by_year.replace(0, 1)
    ).clip(0, 1)
    
    # 4. Temporal normalization
    print("  📅 Computing temporal features...")
    year_range = df_features['year'].max() - df_features['year'].min()
    if year_range > 0:
        df_features['year_normalized'] = (df_features['year'] - df_features['year'].min()) / year_range
    else:
        df_features['year_normalized'] = 0.5
    
    # Fill any NaN values in derived features
    derived_cols = ['driver_skill', 'team_strength', 'race_completion_rate', 'year_normalized']
    for col in derived_cols:
        if col in df_features.columns:
            df_features[col] = df_features[col].fillna(df_features[col].median())
    
    print(f"✅ Created derived features")
    print(f"🔧 Final engineered dataset shape: {df_features.shape}")
else:
    print("⚠️ No data available for derived features.")

🎯 Creating derived features...
  🏆 Computing driver skill ratings...
  🏁 Computing team strength ratings...
  🔧 Computing strategy features...
  📅 Computing temporal features...
✅ Created derived features
🔧 Final engineered dataset shape: (240, 36)


## 🧹 Data Cleaning & Missing Value Treatment

In [6]:
if not df.empty:
    print("🧹 Handling missing values and data cleaning...")
    
    # Get numeric features
    numeric_features = df_features.select_dtypes(include=[np.number]).columns.tolist()
    
    print(f"📊 Processing {len(numeric_features)} numeric features")
    
    # Handle missing values
    missing_summary = df_features[numeric_features].isnull().sum()
    if missing_summary.sum() > 0:
        print(f"\n🔍 Found {missing_summary.sum()} missing values")
        
        # 1. Qualifying times: Fill with median by year
        quali_cols = ['q1_time', 'q2_time', 'q3_time', 'gap_to_pole']
        for col in quali_cols:
            if col in df_features.columns:
                df_features.loc[df_features[col] == 0, col] = np.nan
                df_features[col] = df_features.groupby('year')[col].transform(
                    lambda x: x.fillna(x.median())
                )
                df_features[col] = df_features[col].fillna(df_features[col].median())
        
        # 2. Position features: Fill with median
        position_cols = ['grid_pos', 'finish_pos', 'quali_pos']
        for col in position_cols:
            if col in df_features.columns:
                df_features.loc[df_features[col] == 0, col] = np.nan
                df_features[col] = df_features[col].fillna(df_features[col].median())
        
        # 3. Other features: Fill with median
        for col in numeric_features:
            if col not in quali_cols + position_cols:
                df_features[col] = df_features[col].fillna(df_features[col].median())
    
    # Final check
    remaining_missing = df_features[numeric_features].isnull().sum().sum()
    print(f"✅ Remaining missing values: {remaining_missing}")
    
    if remaining_missing > 0:
        print("🔧 Applying final imputation...")
        imputer = KNNImputer(n_neighbors=3)
        df_features[numeric_features] = imputer.fit_transform(df_features[numeric_features])
        print("✅ Imputation completed")
else:
    print("⚠️ No data available for cleaning.")

🧹 Handling missing values and data cleaning...
📊 Processing 28 numeric features
✅ Remaining missing values: 0


## 📏 Feature Selection & Scaling

In [7]:
if not df.empty:
    print("📏 Feature selection and scaling with importance weighting...")
    
    # Get numeric columns only
    numeric_columns = df_features.select_dtypes(include=[np.number]).columns.tolist()
    target = 'finish_pos'
    
    # Remove target and non-predictive columns (keep encoded features)
    exclude_cols = [target, 'year', 'points', 'finish_pos_binned_encoded']  # Exclude target-derived feature
    available_features = [col for col in numeric_columns if col not in exclude_cols]
    
    print(f"📊 Total numeric columns: {len(numeric_columns)}")
    print(f"📊 Available features (after exclusions): {len(available_features)}")
    print(f"🎯 Target variable: {target}")
    
    # Get weighted features from config
    config_weighted = get_weighted_features("all")
    
    # Map config features to available columns with comprehensive weighting
    feature_weights = {}
    for col in available_features:
        # Check if feature is in config
        if col in config_weighted:
            feature_weights[col] = config_weighted[col]
        # Add weights for engineered features (COMPREHENSIVE)
        elif 'driver_skill' in col or 'team_strength' in col:
            feature_weights[col] = 0.85
        elif 'grid' in col.lower() or 'quali' in col.lower():
            feature_weights[col] = 0.90
        elif 'form' in col.lower() or 'race' in col.lower():
            feature_weights[col] = 0.75  # Rolling features
        elif 'gap' in col.lower() or 'time' in col.lower():
            feature_weights[col] = 0.70  # Timing features
        elif 'pit' in col.lower() or 'strategy' in col.lower():
            feature_weights[col] = 0.65  # Strategy features
        elif 'experience' in col.lower() or 'cumulative' in col.lower():
            feature_weights[col] = 0.60  # Experience features
        elif 'normalized' in col.lower() or 'completion' in col.lower():
            feature_weights[col] = 0.60  # Normalized features
        elif 'encoded' in col.lower():
            feature_weights[col] = 0.70  # Categorical encodings
        elif 'weight' in col.lower():
            feature_weights[col] = 0.75  # Data weighting
        elif 'delta' in col.lower() or 'change' in col.lower():
            feature_weights[col] = 0.65  # Change metrics
        elif 'laps' in col.lower() or 'pos' in col.lower():
            feature_weights[col] = 0.60  # Position/lap features
        else:
            feature_weights[col] = 0.55  # Default weight (included in medium range)
    
    # Create feature sets by importance (more inclusive thresholds)
    high_importance = {k: v for k, v in feature_weights.items() if v >= 0.80}
    medium_importance = {k: v for k, v in feature_weights.items() if 0.60 <= v < 0.80}
    low_importance = {k: v for k, v in feature_weights.items() if 0.50 <= v < 0.60}
    
    print(f"\n⚖️ Feature importance:")
    print(f"  • High (≥0.80): {len(high_importance)} features")
    print(f"  • Medium (0.60-0.80): {len(medium_importance)} features")
    print(f"  • Low (0.50-0.60): {len(low_importance)} features")
    print(f"  • Total weighted: {len(feature_weights)} features")
    
    # Show top weighted features
    top_features = dict(sorted(feature_weights.items(), key=lambda x: x[1], reverse=True)[:15])
    print(f"\n🏆 Top 15 weighted features:")
    for feat, weight in top_features.items():
        print(f"  • {feat:30}: {weight:.2f}")
    
    # Create feature sets
    core_features = list(high_importance.keys())[:10]  # Top 10 high-importance
    
    # VAE features: Use ALL weighted features (high + medium + low)
    vae_features = (list(high_importance.keys()) + 
                   list(medium_importance.keys()) + 
                   list(low_importance.keys()))
    vae_features = list(dict.fromkeys(vae_features))  # Remove duplicates, keep all available
    
    print(f"\n📦 Feature set sizes:")
    print(f"  • Core: {len(core_features)} features")
    print(f"  • VAE: {len(vae_features)} features (all available weighted features)")
    
    feature_sets = {
        'core_weighted': {
            'features': core_features,
            'weights': {f: feature_weights[f] for f in core_features}
        },
        'vae_optimized': {
            'features': vae_features,
            'weights': {f: feature_weights[f] for f in vae_features}
        }
    }
    
    # Prepare datasets
    datasets = {}
    scalers = {}
    
    for set_name, fset in feature_sets.items():
        features = fset['features']
        weights = fset['weights']
        
        if len(features) == 0:
            continue
        
        # Ensure features exist
        existing_features = [f for f in features if f in df_features.columns]
        if len(existing_features) == 0:
            continue
        
        print(f"\n🔧 Preparing {set_name} dataset...")
        print(f"  📊 Using {len(existing_features)} features")
        
        # Extract features and target
        X = df_features[existing_features].copy()
        y = df_features[target].copy()
        
        # Remove rows with missing values
        valid_mask = ~(X.isnull().any(axis=1) | y.isnull())
        X = X[valid_mask]
        y = y[valid_mask]
        
        # Scale features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
        
        # Apply feature weights
        feature_weights_array = np.array([weights.get(col, 0.5) for col in X_scaled.columns])
        X_weighted = X_scaled * feature_weights_array
        
        # Store dataset
        datasets[set_name] = {
            'X': X_weighted,
            'X_scaled': X_scaled,
            'X_raw': X,
            'y': y,
            'features': existing_features,
            'weights': {f: weights[f] for f in existing_features}
        }
        scalers[set_name] = scaler
        
        print(f"  ✅ Shape: X{X_weighted.shape}, y{y.shape}")
    
    print(f"\n✅ Created {len(datasets)} datasets: {list(datasets.keys())}")
else:
    print("⚠️ No data available for feature selection.")

📏 Feature selection and scaling with importance weighting...
📊 Total numeric columns: 28
📊 Available features (after exclusions): 25
🎯 Target variable: finish_pos

⚖️ Feature importance:
  • High (≥0.80): 7 features
  • Medium (0.60-0.80): 13 features
  • Low (0.50-0.60): 1 features
  • Total weighted: 25 features

🏆 Top 15 weighted features:
  • grid_pos                      : 0.95
  • quali_pos                     : 0.90
  • driver_quali_form             : 0.90
  • team_quali_avg                : 0.90
  • quali_race_delta              : 0.90
  • team_strength                 : 0.85
  • driver_skill                  : 0.80
  • gap_to_pole                   : 0.75
  • data_weight                   : 0.75
  • driver_race_form              : 0.75
  • driver_points_form            : 0.75
  • team_race_avg                 : 0.75
  • cumulative_races              : 0.75
  • race_completion_rate          : 0.75
  • gap_to_pole_normalized        : 0.70

📦 Feature set sizes:
  • Core: 7 featur

## 🎲 Filter Invalid Drivers & Create Unbiased Train/Validation Splits

In [8]:
if not df.empty and 'datasets' in locals() and datasets:
    print("🎲 Filtering data and creating unbiased train/validation splits...")
    
    # Step 1: Identify current race year and valid drivers
    print("\n🔍 Step 1: Filtering invalid drivers for current race...")
    current_year = df_features['year'].max()
    print(f"   Current race year: {current_year}")
    
    # Get drivers who participated in the current year
    current_year_drivers = set(df_features[df_features['year'] == current_year]['driver_name'].unique())
    print(f"   Valid drivers in current year: {len(current_year_drivers)}")
    print(f"   Drivers: {sorted(current_year_drivers)}")
    
    # Filter df_features to only include valid drivers
    df_features_filtered = df_features[df_features['driver_name'].isin(current_year_drivers)].copy()
    rows_before = len(df_features)
    rows_after = len(df_features_filtered)
    print(f"   ✅ Filtered dataset: {rows_before} → {rows_after} rows ({rows_before - rows_after} removed)")
    
    # Update datasets with filtered indices
    splits = {}
    
    for set_name, dataset in datasets.items():
        print(f"\n🔄 Processing {set_name} dataset...")
        
        X = dataset['X']
        y = dataset['y']
        
        # Filter to only include valid driver indices
        valid_indices = [idx for idx in X.index if idx in df_features_filtered.index]
        X_filtered = X.loc[valid_indices]
        y_filtered = y.loc[valid_indices]
        
        print(f"   Before filter: {X.shape[0]} samples")
        print(f"   After filter: {X_filtered.shape[0]} samples ({X.shape[0] - X_filtered.shape[0]} removed)")
        
        if len(X_filtered) < 10:
            print(f"   ⚠️ Insufficient data after filtering ({len(X_filtered)} samples)")
            continue
        
        # Step 2: Create UNBIASED stratified split (NO temporal bias)
        print(f"\n   🎯 Creating unbiased stratified split...")
        
        # Stratify by finish position to ensure balanced train/val sets
        y_binned = pd.cut(y_filtered, bins=5, labels=False)
        
        try:
            X_train, X_val, y_train, y_val = train_test_split(
                X_filtered, y_filtered, 
                test_size=0.2,  # 80-20 split
                stratify=y_binned, 
                random_state=42,
                shuffle=True  # Ensure random shuffling
            )
            print(f"   ✅ Stratified split successful")
        except Exception as e:
            print(f"   ⚠️ Stratification failed ({e}), using random split...")
            X_train, X_val, y_train, y_val = train_test_split(
                X_filtered, y_filtered, 
                test_size=0.2, 
                random_state=42,
                shuffle=True
            )
        
        print(f"   📊 Train: {X_train.shape[0]} samples ({X_train.shape[0]/len(X_filtered)*100:.1f}%)")
        print(f"   📊 Val:   {X_val.shape[0]} samples ({X_val.shape[0]/len(X_filtered)*100:.1f}%)")
        
        # Verify year distribution (should be balanced, not temporal)
        train_years = df_features_filtered.loc[X_train.index, 'year']
        val_years = df_features_filtered.loc[X_val.index, 'year']
        
        print(f"\n   📅 Year distribution in train set:")
        train_year_counts = train_years.value_counts().sort_index()
        for year, count in train_year_counts.items():
            print(f"      {year}: {count} samples ({count/len(train_years)*100:.1f}%)")
        
        print(f"   📅 Year distribution in val set:")
        val_year_counts = val_years.value_counts().sort_index()
        for year, count in val_year_counts.items():
            print(f"      {year}: {count} samples ({count/len(val_years)*100:.1f}%)")
        
        # Verify position distribution
        print(f"\n   🏁 Position distribution:")
        print(f"      Train - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}")
        print(f"      Val   - Mean: {y_val.mean():.2f}, Std: {y_val.std():.2f}")
        
        # Store splits (only stratified, no temporal)
        splits[set_name] = {
            'stratified': {
                'X_train': X_train,
                'X_val': X_val,
                'y_train': y_train,
                'y_val': y_val
            },
            'temporal': {  # Keep structure for backward compatibility but set to None
                'X_train': None,
                'X_val': None,
                'y_train': None,
                'y_val': None
            }
        }
    
    print("\n" + "="*70)
    print("✅ UNBIASED SPLIT COMPLETE!")
    print("="*70)
    print(f"✅ Filtered to valid drivers only ({len(current_year_drivers)} drivers)")
    print(f"✅ Created unbiased stratified splits (80-20)")
    print(f"✅ No temporal bias - all years distributed across train/val")
    print(f"✅ Position-balanced - similar distributions in train/val")
else:
    print("⚠️ No datasets available for splitting.")

🎲 Filtering data and creating unbiased train/validation splits...

🔍 Step 1: Filtering invalid drivers for current race...
   Current race year: 2025
   Valid drivers in current year: 21
   Drivers: ['Alexander Albon', 'Carlos Sainz', 'Charles Leclerc', 'Esteban Ocon', 'Fernando Alonso', 'Franco Colapinto', 'Gabriel Bortoleto', 'George Russell', 'Isack Hadjar', 'Jack Doohan', 'Kimi Antonelli', 'Lance Stroll', 'Lando Norris', 'Lewis Hamilton', 'Liam Lawson', 'Max Verstappen', 'Nico Hulkenberg', 'Oliver Bearman', 'Oscar Piastri', 'Pierre Gasly', 'Yuki Tsunoda']
   ✅ Filtered dataset: 240 → 222 rows (18 removed)

🔄 Processing core_weighted dataset...
   Before filter: 240 samples
   After filter: 222 samples (18 removed)

   🎯 Creating unbiased stratified split...
   ✅ Stratified split successful
   📊 Train: 177 samples (79.7%)
   📊 Val:   45 samples (20.3%)

   📅 Year distribution in train set:
      2022: 9 samples (5.1%)
      2023: 12 samples (6.8%)
      2024: 12 samples (6.8%)
     

## 📝 Categorical Encoding

In [9]:
if not df.empty:
    print("📝 Encoding categorical features...")
    
    categorical_encodings = {}
    
    # Driver encoding
    if 'driver_name' in df_features.columns:
        driver_encoder = LabelEncoder()
        df_features['driver_encoded'] = driver_encoder.fit_transform(df_features['driver_name'])
        categorical_encodings['driver'] = {
            'encoder': driver_encoder,
            'classes': driver_encoder.classes_,
            'n_classes': len(driver_encoder.classes_)
        }
        print(f"  🏎️ Encoded {len(driver_encoder.classes_)} drivers")
    
    # Team encoding
    if 'team' in df_features.columns:
        team_encoder = LabelEncoder()
        df_features['team_encoded'] = team_encoder.fit_transform(df_features['team'])
        categorical_encodings['team'] = {
            'encoder': team_encoder,
            'classes': team_encoder.classes_,
            'n_classes': len(team_encoder.classes_)
        }
        print(f"  🏁 Encoded {len(team_encoder.classes_)} teams")
    
    # Year encoding
    year_encoder = LabelEncoder()
    df_features['year_encoded'] = year_encoder.fit_transform(df_features['year'])
    categorical_encodings['year'] = {
        'encoder': year_encoder,
        'classes': year_encoder.classes_,
        'n_classes': len(year_encoder.classes_)
    }
    print(f"  📅 Encoded {len(year_encoder.classes_)} years")
    
    # Finish position binning for Bayesian Network
    position_bins = [0, 5, 10, 15, 21]
    position_labels = ['Podium', 'Points', 'Midfield', 'Backmarker']
    
    df_features['finish_pos_binned'] = pd.cut(
        df_features['finish_pos'], 
        bins=position_bins, 
        labels=position_labels,
        include_lowest=True
    )
    
    pos_bin_encoder = LabelEncoder()
    df_features['finish_pos_binned_encoded'] = pos_bin_encoder.fit_transform(df_features['finish_pos_binned'])
    categorical_encodings['finish_pos_binned'] = {
        'encoder': pos_bin_encoder,
        'classes': pos_bin_encoder.classes_,
        'n_classes': len(pos_bin_encoder.classes_),
        'bins': position_bins,
        'labels': position_labels
    }
    
    print(f"  🎯 Created {len(position_labels)} position categories")
    print(f"\n✅ Categorical encoding complete!")
else:
    print("⚠️ No data available for encoding.")

📝 Encoding categorical features...
  🏎️ Encoded 30 drivers
  🏁 Encoded 13 teams
  📅 Encoded 4 years
  🎯 Created 4 position categories

✅ Categorical encoding complete!


## 💾 Save Preprocessed Data

In [10]:
if not df.empty and 'datasets' in locals() and datasets:
    print("💾 Saving preprocessed data and artifacts...")
    
    os.makedirs('data/preprocessed', exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Determine circuit name
    circuit_name = circuit_name.lower() if 'circuit_name' in locals() else "singapore"
    
    # 1. Save complete engineered dataset (MAIN OUTPUT - Used by notebooks 04 & 05)
    full_dataset_path = f"data/preprocessed/{circuit_name}_engineered_{timestamp}.csv"
    df_features.to_csv(full_dataset_path, index=False)
    print(f"✅ Full engineered dataset: {full_dataset_path}")
    print(f"   📊 Shape: {df_features.shape}")
    print(f"   🎯 This file contains ALL features for VAE and BN models")
    
    # 2. Save preprocessing artifacts (scalers, encoders, splits)
    preprocessing_artifacts = {
        'datasets': datasets,
        'splits': splits if 'splits' in locals() else {},
        'scalers': scalers,
        'categorical_encodings': categorical_encodings if 'categorical_encodings' in locals() else {},
        'feature_sets': feature_sets if 'feature_sets' in locals() else {},
        'circuit_name': circuit_name,
        'timestamp': timestamp,
        'original_shape': df.shape,
        'engineered_shape': df_features.shape
    }
    
    artifacts_path = f"data/preprocessed/preprocessing_artifacts_{circuit_name}_{timestamp}.pkl"
    with open(artifacts_path, 'wb') as f:
        pickle.dump(preprocessing_artifacts, f)
    print(f"✅ Preprocessing artifacts: {artifacts_path}")
    
    # 3. Save individual feature sets (for model-specific usage)
    for set_name, dataset in datasets.items():
        dataset['X'].to_csv(f"data/preprocessed/{circuit_name}_{set_name}_X_{timestamp}.csv")
        dataset['y'].to_csv(f"data/preprocessed/{circuit_name}_{set_name}_y_{timestamp}.csv")
        print(f"  💾 {set_name} dataset saved")
    
    # 4. Save summary with feature list
    summary = {
        'circuit': circuit_name,
        'timestamp': timestamp,
        'original_shape': list(df.shape),
        'engineered_shape': list(df_features.shape),
        'features_created': len(df_features.columns) - len(df.columns),
        'datasets_created': len(datasets),
        'all_features': list(df_features.columns),
        'dataset_info': {
            set_name: {
                'feature_count': len(dataset['features']),
                'sample_count': len(dataset['X']),
                'features': dataset['features']
            } for set_name, dataset in datasets.items()
        }
    }
    
    summary_path = f"data/preprocessed/preprocessing_summary_{circuit_name}_{timestamp}.json"
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)
    print(f"✅ Summary: {summary_path}")
    
    # Final summary
    print("\n" + "="*70)
    print("🎉 PREPROCESSING COMPLETE!")
    print("="*70)
    print(f"🏁 Circuit: {circuit_name.upper()} GP")
    print(f"📊 Original: {df.shape} → Engineered: {df_features.shape}")
    print(f"⭐ Features added: {len(df_features.columns) - len(df.columns)}")
    print(f"🎯 Total features: {len(df_features.columns)}")
    
    print(f"\n📦 Feature Sets Created:")
    for set_name, dataset in datasets.items():
        print(f"  • {set_name}: {len(dataset['features'])} features, {len(dataset['X'])} samples")
    
    print(f"\n🔗 NEXT STEPS:")
    print(f"  1. Notebook 04: Train VAE using engineered dataset")
    print(f"  2. Notebook 05: Train Bayesian Network with VAE latent features")
    print(f"  3. Both notebooks will read from: {full_dataset_path}")
    
    print(f"\n📁 Key Output Files:")
    print(f"  🎯 Main Dataset: {full_dataset_path}")
    print(f"  🔧 Artifacts: {artifacts_path}")
    print(f"  📋 Summary: {summary_path}")
    
    print(f"\n✅ Ready for VAE and Bayesian Network implementation!")
    
else:
    print("❌ Cannot save - no datasets created")

💾 Saving preprocessed data and artifacts...
✅ Full engineered dataset: data/preprocessed/singapore_engineered_20251026_234139.csv
   📊 Shape: (240, 41)
   🎯 This file contains ALL features for VAE and BN models


✅ Preprocessing artifacts: data/preprocessed/preprocessing_artifacts_singapore_20251026_234139.pkl
  💾 core_weighted dataset saved
  💾 vae_optimized dataset saved
✅ Summary: data/preprocessed/preprocessing_summary_singapore_20251026_234139.json

🎉 PREPROCESSING COMPLETE!
🏁 Circuit: SINGAPORE GP
📊 Original: (240, 21) → Engineered: (240, 41)
⭐ Features added: 20
🎯 Total features: 41

📦 Feature Sets Created:
  • core_weighted: 7 features, 240 samples
  • vae_optimized: 21 features, 240 samples

🔗 NEXT STEPS:
  1. Notebook 04: Train VAE using engineered dataset
  2. Notebook 05: Train Bayesian Network with VAE latent features
  3. Both notebooks will read from: data/preprocessed/singapore_engineered_20251026_234139.csv

📁 Key Output Files:
  🎯 Main Dataset: data/preprocessed/singapore_engineered_20251026_234139.csv
  🔧 Artifacts: data/preprocessed/preprocessing_artifacts_singapore_20251026_234139.pkl
  📋 Summary: data/preprocessed/preprocessing_summary_singapore_20251026_234139.json

✅ Rea